In [ ]:
'''Unit test for create_multiple_span_sample()'''
input_ids = \
processor.create_multiple_span_sample(
    lambada[0]['inputs_pretokenized'],
    id_to_completions_ids[0][0], 
    span_length=SPAN_LENGTH, 
    gap_between_spans=GAP_BETWEEN_SPANS, 
    num_spans=NUM_SPANS,
    return_tensor='inputs'
)

labels = \
processor.create_multiple_span_sample(
    lambada[0]['targets_pretokenized'],
    id_to_completions_ids[0][0], 
    span_length=SPAN_LENGTH, 
    gap_between_spans=GAP_BETWEEN_SPANS, 
    num_spans=NUM_SPANS,
    return_tensor='labels'
)

print('inputs_original_string:', lambada[0]['inputs_pretokenized'])
print('labels_original_string:', tokenizer.decode(id_to_completions_ids[0][0]))
print('inputs_original_ids:', torch.tensor(processor.tokenizer(lambada[0]['inputs_pretokenized'])['input_ids']))
print('labels_original_ids:', id_to_completions_ids[0][0])


print('input_ids:', input_ids)
print('labels:', labels)
print('inputs decoded:', tokenizer.decode(input_ids))
print('labels decoded:', tokenizer.decode(labels))

# decode inputs and labels (one by one) in id_span_gap_num_to_input_and_completions
for id_span_gap_num in id_span_gap_num_to_input_and_completions:
    print("\nid_span_gap_num:\n", id_span_gap_num)
    print('inputs_original_string:', tokenizer.decode(tokenizer(lambada[id_span_gap_num[0]]['inputs_pretokenized']).input_ids))
    print("\ninputs", tokenizer.decode(id_span_gap_num_to_input_and_completions[id_span_gap_num]['inputs']))

    for i in range(len(id_span_gap_num_to_input_and_completions[id_span_gap_num]['labels'])):
        print("\ncompletion:", tokenizer.decode(id_span_gap_num_to_input_and_completions[id_span_gap_num]['labels'][i]))
        print('labels_original_string:', tokenizer.decode(id_to_completions_ids[id_span_gap_num[0]][i]))
    print("")

 

In [ ]:
'''tokenizer vocab, decode, encode'''
vocab = tokenizer.get_vocab() 
# sort by key
vocab = {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1])}
print(vocab)

tokenizer('``') == <unk>

In [ ]:
'''align get_multiple_span_samples() and get_middle_off_samples()'''

assert torch.all(id_span_gap_num_to_input_and_completions[(0,3,5,1)]['inputs'] == id_middlespan_gap_to_input_and_completions[(0,3,5)]['inputs']).tolist()
assert torch.all(id_span_gap_num_to_input_and_completions[(0,3,5,1)]['labels'][0] == id_middlespan_gap_to_input_and_completions[(0,3,5)]['labels'][0]).tolist()

for key in id_middlespan_gap_to_input_and_completions:
    new_key = (key[0], key[1], key[2], 1)
    # assert inputs
    assert torch.all(id_span_gap_num_to_input_and_completions[new_key]['inputs'] == id_middlespan_gap_to_input_and_completions[key]['inputs']).tolist()
    # assert labels
    for i in range(len(id_span_gap_num_to_input_and_completions[new_key]['labels'])):
        assert torch.all(id_span_gap_num_to_input_and_completions[new_key]['labels'][i] == id_middlespan_gap_to_input_and_completions[key]['labels'][i]).tolist()

In [ ]:
'''Obtain the avg_log_p_map for middle-off samples via data parallelism'''
RUN_CELL = 0
if RUN_CELL:
    from multiprocessing import Process
    import multiprocessing
    avg_log_p_map_middle_off = dict()
    # define the processing for each id_middlespan_gap example as a function and use threading to use 3 models in parallel
    def process(list_id_middlespan_gap, model_, device='cuda:0'):
        for id_middlespan_gap in tqdm(list_id_middlespan_gap):
            input_ids = id_middlespan_gap_to_input_and_completions[id_middlespan_gap]['inputs'].unsqueeze(0).to(device)
            completions_batch = id_middlespan_gap_to_input_and_completions[id_middlespan_gap]['labels'].to(device)
            outputs = lambada_utils.multi_labels_forward(model_, input_ids, completions_batch)

            middlespan_length = id_middlespan_gap[1]

            for completion_index in range(len(completions_batch)):
                avg_log_p = -ce_loss(
                    # Only care about the tokens corresponding to the last word and omit offset tokens 
                    # the first one is <extra_id_0> and omitted
                    outputs.logits[completion_index][2+middlespan_length:], 
                    completions_batch[completion_index][2+middlespan_length:]
                )
                avg_log_p_map_middle_off[(*id_middlespan_gap, completion_index)] = \
                    avg_log_p.detach().cpu().tolist()
            
    # run the above function in parallel
    import threading
    multiprocessing.set_start_method('spawn')

    all_id_middlespan_gaps = list(id_middlespan_gap_to_input_and_completions.keys())
    all_id_middlespan_gaps_0 = all_id_middlespan_gaps[:len(all_id_middlespan_gaps)//2]
    all_id_middlespan_gaps_1 = all_id_middlespan_gaps[len(all_id_middlespan_gaps)//2:]
    # all_id_middlespan_gaps_2 = all_id_middlespan_gaps[2*len(all_id_middlespan_gaps)//3:]

    t0 = Process(target=process, args=(all_id_middlespan_gaps_0, model, 'cuda:0'))
    t1 = Process(target=process, args=(all_id_middlespan_gaps_1, model1, 'cuda:2'))
    # t2 = threading.Thread(target=process, args=(all_id_middlespan_gaps_2, model2, 'cuda:4'))

    t0.start()
    t1.start()
    # t2.start()

In [ ]:
RUN_CELL = 0 # '''Generate the top completions (through beam search) for each example, and get the word from each completion.'''
if RUN_CELL:
    # generate for all examples, and then get the words from the completions, and compare the first one with the target
    count_correct = 0 # No. correct last word predictions if only the top completion is considered
    count_correct_top_num_beams = 0 # ... if the top num_beams completions are considered
    count_no_words_found = 0  # No. examples where no valid last word is found

    # punctuated_word: the last word and the punctuation that follows it
    id_to_punctuated_words = {} # maps example index to a list of word and punc pairs; every punc is kept for each word
    id_to_punctuated_words_unique = {} # ...; every punc is kept for each word  
    id_to_completions_ids = {}

    MAX_COMPLETION_LENGTH = 8 # for last word prediction, 8 is sufficient
    NUM_BEAMS = 20 # 20 is sufficient; more doesn't help

    # for example_index in tqdm(range(10)): # len(lambada)
    for example_index in tqdm(range(len(lambada))): # len(lambada)
        input_string = lambada[example_index]['inputs_pretokenized']
        inputs = tokenizer(input_string, return_tensors="pt").input_ids.to("cuda")
        outputs = model.generate(inputs,
                                max_length=MAX_COMPLETION_LENGTH, 
                                num_beams=NUM_BEAMS, 
                                num_return_sequences=NUM_BEAMS, 
                                output_scores=True,
                                eos_token_id=tokenizer.convert_tokens_to_ids('<extra_id_1>'), 
                                return_dict_in_generate=True)
        
        completions = [tokenizer.decode(outputs['sequences'][i]) for i in range(NUM_BEAMS)]
        completions_ids_with_valid_lastword = [
            outputs['sequences'][i].cpu()
            for i in range(NUM_BEAMS)
            if processor.get_word_from_completion(completions[i]) is not None # if the completion has a valid last word
        ]

        words = processor.get_words_from_completions(completions)
        words_unique = list(set(words))

        # TODO: combine them and move to lambada_utils.py
        completions_without_pad = processor.remove_pad_id(completions_ids_with_valid_lastword)
        completions_without_pad_before_punctuation = processor.before_first_punc_including(completions_without_pad) # including the first punc
        
        if words:
            if words[0] == lambada[example_index]['targets_pretokenized'][0]:
                count_correct += 1
        else:
            count_no_words_found += 1
            # print("no words found")
        punctuated_words = processor.get_punctuated_words(completions)
        id_to_punctuated_words[example_index] = punctuated_words
        
        completions_without_pad_before_punctuation_unique = list(set([tuple(x.numpy()) for x in completions_without_pad_before_punctuation]))
        id_to_completions_ids[example_index] = [torch.tensor(x) for x in completions_without_pad_before_punctuation_unique]

        # find the best punctuatuation for each unique word (Maximum Probability Strategy, 
        # completions are naturally ordered by probs by generate()) TODO: move this for loop to utils.py
        id_to_punctuated_words_unique[example_index] = []
        for word in words_unique:
            found = 0
            # iterate through the word and punc pairs, and find the one that matches the word
            for punctuated_word in punctuated_words:
                # it is a match if pair = word + punc
                ENDING_PUNCTUATIONS = ',!.:;?'
                for punc in ENDING_PUNCTUATIONS:
                    if punctuated_word == word + punc:
                        id_to_punctuated_words_unique[example_index].append(punctuated_word)
                        found = 1
                        break
                if found == 1:
                    break
        
        # calculate the number of correct top num_beams: if the correct word is in the top num_beams, then it is correct
        for word in words_unique:
            if word == lambada[example_index]['targets_pretokenized'][0]:
                count_correct_top_num_beams += 1
                break
    print("count_correct", count_correct)
    # count_correct, NLU: 0.7595
    # count_correct, NLG: 0.7680 (test)
    # count_correct, NLG: 0.7689 (test, spaced punc fixed)
    # count_correct, S2S: 0.3743 (could be because how the mode handles extra_ids)


In [ ]:
RUN_CELL = 0 # TODO: a lot of duplicates in id_to_completions_ids, remove them
if RUN_CELL:
    id_to_completions_ids_unique = {}
    for key in id_to_completions_ids:
        completions_ids_unique = list(set([tuple(x.numpy()) for x in id_to_completions_ids[key]]))
        id_to_completions_ids_unique[key] = [torch.tensor(x) for x in completions_ids_unique]
    ul2_lambada_vanilla_beam_search_results['id_to_completions_ids'] = id_to_completions_ids_unique
    # save it back to the pickle file
    with open(timed_pickle_filename, 'wb') as fp:
        pickle.dump(ul2_lambada_vanilla_beam_search_results, fp)